In [3]:
import torch
import torch.nn as nn 
from torch.autograd import Variable
import pennylane as qml
from pennylane import numpy as np
import math 
from sympy import *
import torchvision
from torch.utils.data import Dataset, DataLoader, random_split
from PeptideDataset import *

In [4]:
#assuming r=1,
def cartesian_to_spherical(x, y, z):
    theta = math.acos(z / 1)  
    phi = math.atan2(y, x)
    if phi < 0:
        phi += 2*np.pi
    return [theta, phi]


    


def fibonacci_sphere(n):

    points = []
    phi = math.pi * (math.sqrt(5.) - 1.)  # golden angle in radians

    for i in range(n):
        y = 1 - (i / float(n - 1)) * 2  # y goes from 1 to -1
        radius = math.sqrt(1 - y * y)  # radius at y

        theta = phi * i  # golden angle increment

        x = math.cos(theta) * radius
        z = math.sin(theta) * radius
        points.append((x, y, z))
        sph_points = [cartesian_to_spherical(x,y,z) for x,y,z in points]
    
    return sph_points



In [5]:
dtype = torch.cuda.DoubleTensor if torch.cuda.is_available() else torch.DoubleTensor
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class VariationalQuantumClassifierInterface:
    def __init__(
            self,
            num_of_input,
            num_of_output,
            num_of_wires,
            num_of_layers,
            var_Q_circuit,
            var_Q_bias,
            qdevice):

        self.var_Q_circuit = var_Q_circuit
        self.var_Q_bias = var_Q_bias
        self.num_of_input = num_of_input
        self.num_of_output = num_of_output
        self.num_of_wires = num_of_wires
        self.num_of_layers = num_of_layers

        self.qdevice = qdevice

        self.dev = qml.device(self.qdevice, wires = num_of_wires)


    def set_params(self, var_Q_circuit, var_Q_bias):
        self.var_Q_circuit = var_Q_circuit
        self.var_Q_bias = var_Q_bias

    def init_params(self):
        self.var_Q_circuit = Variable(torch.tensor(0.01 * np.random.randn(self.num_of_layers, self.num_of_wires, 3), device=device).type(dtype), requires_grad=True)
        return self.var_Q_circuit

    def _statepreparation(self, angles):

        """Encoding block of circuit given angles

        Args:
            a: feature vector of rad and rad_square => np.array([rad_X_0, rad_X_1, rad_square_X_0, rad_square_X_1])
        """
        # num_of_input determines the number of rotation needed.

        for i in range(self.num_of_input):
            qml.RY(angles[i,0], wires=i)
            qml.RZ(angles[i,1], wires=i)

    def _layer(self, W):
        """ Single layer of the variational classifier.

        Args:
            W (array[float]): 2-d array of variables for one layer

        """

        # Entanglement Layer

        for i in range(self.num_of_wires):
            qml.CNOT(wires=[i, (i + 1) % self.num_of_wires])

        # Rotation Layer
        for j in range(self.num_of_wires):
            qml.Rot(W[j, 0], W[j, 1], W[j, 2], wires=j)

    def circuit(self, angles):

        @qml.qnode(self.dev, interface='torch', diff_method = "parameter-shift")
        def _circuit(var_Q_circuit, angles):
            """The circuit of the variational classifier."""
            self._statepreparation(angles)
            weights = var_Q_circuit

            for W in weights:
                self._layer(W)


            k = self.num_of_input-1
            return [qml.expval(qml.PauliZ(k))]

        return _circuit(self.var_Q_circuit, angles)



    def forward(self, angles):
        result = ((self.circuit(angles).item()))
        return torch.tensor(result, requires_grad=True)

In [6]:
vqc = VariationalQuantumClassifierInterface(
            num_of_input =6,
            num_of_output =1,
            num_of_wires=6,
            num_of_layers=2,
            var_Q_circuit=None,
            var_Q_bias = None,
            qdevice = "default.qubit")
           
fib_angles = fibonacci_sphere(18)
         

class VQCTorch(nn.Module):
    def __init__(self):
        super().__init__()

        self.q_params = nn.Parameter(0.01 * torch.randn(2, 12, 3))
    def get_angles(self, in_x):
        in_x_int = [int(item) for item in in_x.tolist()]
        angles = []
        for item in in_x_int:
            theta = fib_angles[item][0]
            phi = fib_angles[item][1]
            angles.append([theta, phi])

        return torch.tensor(angles, requires_grad=True)


    def forward(self, batch_item):
        vqc.var_Q_circuit = self.q_params
        output_batch = []

        for single_item in batch_item:
            angles = self.get_angles(single_item)

            q_out_elem = vqc.forward(angles)
            
            output_batch.append(q_out_elem)

        outputs = torch.stack(output_batch).view(len(batch_item), 1) 
        return outputs


In [7]:
params = {'batch_size' : 4, 'lr': 0.01, 'epochs': 100}

In [8]:
 

dataset = ToyPeptideDataset()

train_size = int(0.8 * len(dataset))
test_size = (len(dataset) - train_size) // 2
val_size = len(dataset) - train_size - test_size

train_dataset, test_dataset, val_dataset = random_split(dataset, [train_size, test_size, val_size])

print(f"Dataset: {len(dataset)}")
print(f"Train: {len(train_dataset)}")
print(f"Test: {len(test_dataset)}")
print(f"Validation: {len(val_dataset)}")

batch_size = params['batch_size']

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

train_iter = iter(train_loader)
train_data = next(train_iter)
x_train, y_train = train_data

test_iter = iter(test_loader)
test_data = next(test_iter)
x_test, y_test = test_data

val_iter = iter(val_loader)
val_data = next(val_iter)
x_val, y_val = val_data


                          

Dataset: 95207
Train: 76165
Test: 9521
Validation: 9521


In [9]:
model = VQCTorch()

lr = params['lr']

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)  

batch_size = params['batch_size']
num_epochs = params['epochs']
total_samples = len(train_dataset)
n_iterations = math.ceil(total_samples/batch_size)
for epoch in range(num_epochs):
    print(f"EPOCH: {epoch}")
    for i, (data, target) in enumerate(train_loader):
        if i == 0:
            print(f'Inputs {data.shape} | Labels {target.shape}')
        y_predicted = model(data)
        loss = criterion(y_predicted, target)
    
        loss.backward()
        optimizer.step()

        optimizer.zero_grad()

        #
        if (i+1) % 5 == 0:
            print(f'Epoch: {epoch+1}/{num_epochs}, Step {i+1}/{n_iterations}|loss = {loss.item():.4f}')


        
        



EPOCH: 0
Inputs torch.Size([4, 6]) | Labels torch.Size([4, 1])
Epoch: 1/100, Step 5/19042|loss = 0.5096
Epoch: 1/100, Step 10/19042|loss = 0.3745
Epoch: 1/100, Step 15/19042|loss = 0.0926
Epoch: 1/100, Step 20/19042|loss = 0.5275
Epoch: 1/100, Step 25/19042|loss = 0.4457
Epoch: 1/100, Step 30/19042|loss = 0.1663
Epoch: 1/100, Step 35/19042|loss = 0.2461
Epoch: 1/100, Step 40/19042|loss = 0.2919
Epoch: 1/100, Step 45/19042|loss = 0.3937
Epoch: 1/100, Step 50/19042|loss = 0.3327
Epoch: 1/100, Step 55/19042|loss = 0.4958
Epoch: 1/100, Step 60/19042|loss = 0.4808
Epoch: 1/100, Step 65/19042|loss = 0.5933
Epoch: 1/100, Step 70/19042|loss = 0.5418
Epoch: 1/100, Step 75/19042|loss = 0.4061
Epoch: 1/100, Step 80/19042|loss = 0.7188
Epoch: 1/100, Step 85/19042|loss = 0.2429
Epoch: 1/100, Step 90/19042|loss = 0.5258
Epoch: 1/100, Step 95/19042|loss = 0.4869
Epoch: 1/100, Step 100/19042|loss = 0.3609
Epoch: 1/100, Step 105/19042|loss = 0.1659
Epoch: 1/100, Step 110/19042|loss = 0.4897
Epoch: 1/10

Epoch: 1/100, Step 960/19042|loss = 0.4772
Epoch: 1/100, Step 965/19042|loss = 0.1246
Epoch: 1/100, Step 970/19042|loss = 0.4040
Epoch: 1/100, Step 975/19042|loss = 0.2885
Epoch: 1/100, Step 980/19042|loss = 0.5032
Epoch: 1/100, Step 985/19042|loss = 0.2809
Epoch: 1/100, Step 990/19042|loss = 0.4352
Epoch: 1/100, Step 995/19042|loss = 0.1798
Epoch: 1/100, Step 1000/19042|loss = 0.5060
Epoch: 1/100, Step 1005/19042|loss = 0.6500
Epoch: 1/100, Step 1010/19042|loss = 0.3518
Epoch: 1/100, Step 1015/19042|loss = 0.1369
Epoch: 1/100, Step 1020/19042|loss = 0.2672
Epoch: 1/100, Step 1025/19042|loss = 0.5000
Epoch: 1/100, Step 1030/19042|loss = 0.3650
Epoch: 1/100, Step 1035/19042|loss = 0.3211
Epoch: 1/100, Step 1040/19042|loss = 0.6406
Epoch: 1/100, Step 1045/19042|loss = 0.4288
Epoch: 1/100, Step 1050/19042|loss = 0.2606
Epoch: 1/100, Step 1055/19042|loss = 0.3370
Epoch: 1/100, Step 1060/19042|loss = 0.7709
Epoch: 1/100, Step 1065/19042|loss = 0.7031
Epoch: 1/100, Step 1070/19042|loss = 0.3

Epoch: 1/100, Step 1900/19042|loss = 0.7199
Epoch: 1/100, Step 1905/19042|loss = 0.5076
Epoch: 1/100, Step 1910/19042|loss = 0.1981
Epoch: 1/100, Step 1915/19042|loss = 0.3392
Epoch: 1/100, Step 1920/19042|loss = 0.4274
Epoch: 1/100, Step 1925/19042|loss = 0.5099
Epoch: 1/100, Step 1930/19042|loss = 0.4935
Epoch: 1/100, Step 1935/19042|loss = 0.2816
Epoch: 1/100, Step 1940/19042|loss = 0.1895
Epoch: 1/100, Step 1945/19042|loss = 0.7736
Epoch: 1/100, Step 1950/19042|loss = 0.6133
Epoch: 1/100, Step 1955/19042|loss = 0.3293
Epoch: 1/100, Step 1960/19042|loss = 0.3506
Epoch: 1/100, Step 1965/19042|loss = 0.2327
Epoch: 1/100, Step 1970/19042|loss = 0.3662
Epoch: 1/100, Step 1975/19042|loss = 0.4077
Epoch: 1/100, Step 1980/19042|loss = 0.4559
Epoch: 1/100, Step 1985/19042|loss = 0.5376
Epoch: 1/100, Step 1990/19042|loss = 0.4666
Epoch: 1/100, Step 1995/19042|loss = 0.7648
Epoch: 1/100, Step 2000/19042|loss = 0.3046
Epoch: 1/100, Step 2005/19042|loss = 0.2802
Epoch: 1/100, Step 2010/19042|lo

Epoch: 1/100, Step 2835/19042|loss = 0.3133
Epoch: 1/100, Step 2840/19042|loss = 0.2915
Epoch: 1/100, Step 2845/19042|loss = 0.4909
Epoch: 1/100, Step 2850/19042|loss = 0.7081
Epoch: 1/100, Step 2855/19042|loss = 0.4196
Epoch: 1/100, Step 2860/19042|loss = 0.3484
Epoch: 1/100, Step 2865/19042|loss = 0.1788
Epoch: 1/100, Step 2870/19042|loss = 0.1999
Epoch: 1/100, Step 2875/19042|loss = 0.4447
Epoch: 1/100, Step 2880/19042|loss = 0.2410
Epoch: 1/100, Step 2885/19042|loss = 0.3754
Epoch: 1/100, Step 2890/19042|loss = 0.2755
Epoch: 1/100, Step 2895/19042|loss = 0.3693
Epoch: 1/100, Step 2900/19042|loss = 0.3068
Epoch: 1/100, Step 2905/19042|loss = 0.2625
Epoch: 1/100, Step 2910/19042|loss = 0.4575
Epoch: 1/100, Step 2915/19042|loss = 0.1633
Epoch: 1/100, Step 2920/19042|loss = 0.5767
Epoch: 1/100, Step 2925/19042|loss = 0.3366
Epoch: 1/100, Step 2930/19042|loss = 0.2741
Epoch: 1/100, Step 2935/19042|loss = 0.3250
Epoch: 1/100, Step 2940/19042|loss = 0.4213
Epoch: 1/100, Step 2945/19042|lo

Epoch: 1/100, Step 3770/19042|loss = 0.5386
Epoch: 1/100, Step 3775/19042|loss = 0.3787
Epoch: 1/100, Step 3780/19042|loss = 0.3096
Epoch: 1/100, Step 3785/19042|loss = 0.2391
Epoch: 1/100, Step 3790/19042|loss = 0.4602
Epoch: 1/100, Step 3795/19042|loss = 0.2293
Epoch: 1/100, Step 3800/19042|loss = 0.5802
Epoch: 1/100, Step 3805/19042|loss = 0.1787
Epoch: 1/100, Step 3810/19042|loss = 0.2669
Epoch: 1/100, Step 3815/19042|loss = 0.6391
Epoch: 1/100, Step 3820/19042|loss = 0.3316
Epoch: 1/100, Step 3825/19042|loss = 0.3514
Epoch: 1/100, Step 3830/19042|loss = 0.1168
Epoch: 1/100, Step 3835/19042|loss = 0.2737
Epoch: 1/100, Step 3840/19042|loss = 0.1028
Epoch: 1/100, Step 3845/19042|loss = 0.2755
Epoch: 1/100, Step 3850/19042|loss = 0.3311
Epoch: 1/100, Step 3855/19042|loss = 0.3637
Epoch: 1/100, Step 3860/19042|loss = 0.5588
Epoch: 1/100, Step 3865/19042|loss = 0.1677
Epoch: 1/100, Step 3870/19042|loss = 0.1857
Epoch: 1/100, Step 3875/19042|loss = 0.3221
Epoch: 1/100, Step 3880/19042|lo

Epoch: 1/100, Step 4710/19042|loss = 0.2133
Epoch: 1/100, Step 4715/19042|loss = 0.5298
Epoch: 1/100, Step 4720/19042|loss = 0.4235
Epoch: 1/100, Step 4725/19042|loss = 0.1021
Epoch: 1/100, Step 4730/19042|loss = 0.3401
Epoch: 1/100, Step 4735/19042|loss = 0.7577
Epoch: 1/100, Step 4740/19042|loss = 0.5694
Epoch: 1/100, Step 4745/19042|loss = 0.1281
Epoch: 1/100, Step 4750/19042|loss = 0.2347
Epoch: 1/100, Step 4755/19042|loss = 0.6312
Epoch: 1/100, Step 4760/19042|loss = 0.4395
Epoch: 1/100, Step 4765/19042|loss = 0.6320
Epoch: 1/100, Step 4770/19042|loss = 0.5831
Epoch: 1/100, Step 4775/19042|loss = 0.5308
Epoch: 1/100, Step 4780/19042|loss = 0.2394
Epoch: 1/100, Step 4785/19042|loss = 0.2936
Epoch: 1/100, Step 4790/19042|loss = 0.2743
Epoch: 1/100, Step 4795/19042|loss = 0.5049
Epoch: 1/100, Step 4800/19042|loss = 0.3962
Epoch: 1/100, Step 4805/19042|loss = 0.2904
Epoch: 1/100, Step 4810/19042|loss = 0.4366
Epoch: 1/100, Step 4815/19042|loss = 0.2227
Epoch: 1/100, Step 4820/19042|lo

KeyboardInterrupt: 